# Analiza korisnika i zadataka na platformi RingZer0

In [43]:
import pandas as pd
import requests

# Prikupljanje podataka

Prvo želimo prikupiti sve kategorije u RingZero i prebaciti ih u standardne jeopardy kategorije.

In [58]:
# Prikupi info o kategorijama 
url = "https://ringzer0ctf.com/api/categories"
response = requests.get(url)
data = response.json()

# Prikupi sve category id-jeve
categories = data['data']['categories']
category_ids = [cat['category']['id'] for cat in categories]
category_titles = [cat['category']['title'] for cat in categories]
print(f"Found {len(category_ids)} categories: {category_ids}")  
print(f"Category titles: {category_titles}")

Found 21 categories: ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21']
Category titles: ['SQL Injection', 'Reverse Engineering', 'Steganography', 'Forensics', 'Coding Challenges', 'Cryptography', 'JavaScript', 'Web', 'Pwnage Linux', 'SysAdmin Linux', 'Jail Escaping', 'Shellcoding', 'Malware Analysis', 'The NC8 Reverse Engineering Track', 'Exotic Data Storage', 'Software Defined Radio', 'NorthSec 2021', 'NorthSec 2023', 'NorthSec 2022', 'NorthSec 2024', 'NorthSec 2025']


In [59]:
category_ids = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21']
category_titles = ['SQL Injection', 'Reverse Engineering', 'Steganography', 'Forensics', 'Coding Challenges', 'Cryptography', 'JavaScript', 'Web', 'Pwnage Linux', 'SysAdmin Linux', 'Jail Escaping', 'Shellcoding', 'Malware Analysis', 'The NC8 Reverse Engineering Track', 'Exotic Data Storage', 'Software Defined Radio', 'NorthSec 2021', 'NorthSec 2023', 'NorthSec 2022', 'NorthSec 2024', 'NorthSec 2025']

categories = {"Web": ["SQL Injection", "JavaScript", "Web"],
              "Rev": ["Reverse Engineering", "The NC8 Reverse Engineering Track"], 
              "Pwn": ["Pwnage Linux", "Jail Escaping", "Shellcoding", "Malware Analysis"], 
              "Crypt": ["Cryptography", ], 
              "Forensics": ["Forensics", "Exotic Data Storage", "Steganography", "Coding Challenges"], 
              "Misc": ["Software Defined Radio"], 
              "Special": ["NorthSec 2021", "NorthSec 2022", "NorthSec 2023", "NorthSec 2024", "NorthSec 2025"]}

Priprema tablica za podatke

In [ ]:
# Tables content
# users_df = pd.DataFrame(columns=["id", "username", "country", "joined_date", "first_date", "last_date"])
# challenges_df = pd.DataFrame(columns=["category_id", "category_title", "challenge_id", "challenge_title", "points"])
# solves_df = pd.DataFrame(columns=["user_id", "challenge_id", "solve_date"])

In [ ]:
challenge_list = []
for cat_id in category_ids:
    url = f"https://ringzer0ctf.com/api/category/challenges/{cat_id}"
    response = requests.get(url)
    cat_data = response.json()['data']['categories'][0]['category']
    for challenge in cat_data['challenges']:
        challenge_row = {
            'category_id': cat_id,
            'category_title': cat_data['title'],
            'challenge_id': challenge['challenge']['id'],
            'challenge_title': challenge['challenge']['title'],
            'challenge_points': challenge['challenge']['points']
        }
        challenge_list.append(challenge_row)
    
challenges_df = pd.DataFrame(challenge_list)
# challenges_df.to_csv('challenges.csv', index=False)
print(f"Challenges saved to challenges.csv ({len(challenge_list)} challenges total)")
print(challenges_df.head())

KeyboardInterrupt: 

In [69]:
# učitati zadatke iz challenge.csv
challenges_df = pd.read_csv('challenges.csv')
#Dohvatiti sve ideve zadataka
challenge_ids = challenges_df['challenge_id'].tolist()

In [70]:
# Prikupi info o svakom zadatku 300 poziva
solve_nums = {}
users_set = set()
solves = []
for id in challenge_ids:
    url = f"https://ringzer0ctf.com/api/challenge/users/{id}"
    response = requests.get(url)
    solve = response.json()['data']
    # za broj rjesenja u challnges_df
    solve_nums[id] = len(solve['users'])
    # za stvaranje users_df
    users_set.update([(u['user']['id'], u['user']['username'], u['user']['isRCEH'], u['user']['country']) for u in solve['users']])
    # za povezivanje korisnika i rjesenja zadataka
    solvers = solve['categories'][0]['category']['challenges'][0]['challenge']['Solvers']
    # u apiju pise validation_time, ali je zapravo validationTime
    solves.extend([{
        "challenge_id": id,
        "user_id": s['Solver']['userId'],
        "validation_time": s['Solver']['validationTime']
    } for s in solvers])

In [71]:
users_df = pd.DataFrame(list(users_set), columns=["id", "username", "isRCEH", "country"])
users_df.head()

,id,username,isRCEH,country
0,44892,khainguyen,False,Viet Nam
1,27022,Orgish,False,United States
2,18150,glofdd,False,Thailand
3,17771,logon13,False,New Zealand
4,140,3des,False,Canada


In [72]:
challenges_df["challenge_solves"] = challenges_df['challenge_id'].map(solve_nums)
challenges_df.head()


,category_id,category_title,challenge_id,challenge_title,challenge_points,challenge_solves
0,1,SQL Injection,1,Most basic SQLi pattern.,1,10789
1,1,SQL Injection,2,ACL rulezzz the world.,2,5323
2,1,SQL Injection,3,Login portal 1,2,4211
3,1,SQL Injection,171,Random Login Form,2,2581
4,1,SQL Injection,21,Po po po po postgresql,2,2632


In [73]:
solve_df = pd.DataFrame(solves)
solve_df.head()

,challenge_id,user_id,validation_time
0,1,52826,2025-08-29 15:32:22
1,1,52806,2025-08-26 12:23:04
2,1,52805,2025-08-26 11:56:27
3,1,52613,2025-08-22 09:42:06
4,1,52721,2025-08-14 07:47:19


In [74]:
solve_df.to_csv('solves.csv', index=False)
users_df.to_csv('users.csv', index=False)
challenges_df.to_csv('challenges1.csv', index=False)

# Analiza podataka

In [67]:
challenges_df = pd.read_csv('challenges1.csv')
users_df = pd.read_csv('users.csv')
solve_df = pd.read_csv('solves.csv')

Prosječna riješenost po kategoriji za sve korisnike

In [75]:
challenges_df.head()

,category_id,category_title,challenge_id,challenge_title,challenge_points,challenge_solves
0,1,SQL Injection,1,Most basic SQLi pattern.,1,10789
1,1,SQL Injection,2,ACL rulezzz the world.,2,5323
2,1,SQL Injection,3,Login portal 1,2,4211
3,1,SQL Injection,171,Random Login Form,2,2581
4,1,SQL Injection,21,Po po po po postgresql,2,2632


In [76]:
avg_solve_rate_per_category = challenges_df.groupby('category_title')['challenge_solves'].mean().reset_index()

In [77]:
print(avg_solve_rate_per_category)

                       category_title  challenge_solves
0                   Coding Challenges        984.529412
1                        Cryptography        744.555556
2                 Exotic Data Storage        438.000000
3                           Forensics        773.607143
4                       Jail Escaping        467.714286
5                          JavaScript       2767.300000
6                    Malware Analysis        117.090909
7                       NorthSec 2021         49.100000
8                       NorthSec 2022         13.750000
9                       NorthSec 2023         15.148148
10                      NorthSec 2024          5.714286
11                      NorthSec 2025          0.380952
12                       Pwnage Linux        136.142857
13                Reverse Engineering        209.089286
14                      SQL Injection       1053.031250
15                        Shellcoding        230.714286
16             Software Defined Radio        252

In [78]:
# scatterplot
import matplotlib.pyplot as plt

selected_categories = category_titles[:-5]
filtered_df = challenges_df[challenges_df['category_title'].isin(selected_categories)]
filtered_df.head()


,category_id,category_title,challenge_id,challenge_title,challenge_points,challenge_solves
0,1,SQL Injection,1,Most basic SQLi pattern.,1,10789
1,1,SQL Injection,2,ACL rulezzz the world.,2,5323
2,1,SQL Injection,3,Login portal 1,2,4211
3,1,SQL Injection,171,Random Login Form,2,2581
4,1,SQL Injection,21,Po po po po postgresql,2,2632


Prosječna riješenost po kategoriji za 90. centil

Prosječna riješenost unutar kategorije za sve korisnike